before running this, you should run make_tagcomplete_csv.ipynb

In [ ]:
import re
import sys

# get character info from character.txt
file = "downloads/character.txt"
character_dict = {}
with open(file, 'r') as infile:
    for row in infile:
        _, character, _ = row.replace('\\', '').split(', ')
        character_dict[character.strip().replace(' ', '_')] = {}
infile.close()
character_list = list(character_dict.keys())

In [ ]:
unnecessary_tag_pattern = [
    '_quality',
    'commentary',
    'translat',
    '_background',
    'looking_'
]

unnecessary_tags = [
    'highres',
    'solo',
    'newest',
    'late',
    'mid',
    'absurdres',
    'nsfw',
    'comic',
    'upper_body',
    'full_body',
    'standing',
    'bad_id',
    'bad_pixiv_id',
    'bad_twitter_id',
    'sweat',
    'hetero',
    'censored',
    'indoor',
    'outdoor',
    'smile',
    'blush'
]

# read data from csv
file = "intermediates/taglist.csv"
with open(file, 'r') as infile:
    for row in infile:
        tags = row.strip().split(', ')
        characters = [];
        after_rating = False;
        for tag in tags:
            tag = tag.replace('||| ', '').strip().replace(' ', '_')
            
            if tag in character_list:
                characters.append(tag)
            elif after_rating:
                if (not any(pattern in tag for pattern in unnecessary_tag_pattern) and
                    not any(unnecessary_tag == tag for unnecessary_tag in unnecessary_tags)):
                    for character in characters:
                        if tag not in character_dict[character]:
                            character_dict[character][tag] = 0
                        character_dict[character][tag] += 1
            
            if "rating:" in tag:
                after_rating = True
infile.close()

In [ ]:
TOTAL_ADDITIONAL_TAGS = 10

cut_dict = {}
for key, value in character_dict.items():
    ordered_value = dict(sorted(value.items(), key=lambda x: x[1], reverse=True))
    cut_dict[key] = dict(list(ordered_value.items())[:TOTAL_ADDITIONAL_TAGS])

file = "downloads/character.txt"
newfile = "outputs/character_plus.txt"
with open(file, 'r') as infile:
    with open(newfile, 'w') as outfile:
        for row in infile:
            _, character, _ = row.replace('\\', '').split(', ')
            outfile.write(row.strip())
            if character.strip().replace(' ', '_') in cut_dict:
                for key in cut_dict[character.strip().replace(' ', '_')].keys():
                    formatted = key.replace('(', '\\(').replace(')','\\)').replace('_', ' ')
                    outfile.write(f", {formatted}")
            outfile.write("\n")
    outfile.close()
infile.close()